## 数据处理

In [3]:
import pandas as pd
import io
import re
from datetime import datetime, timedelta
from typing import List, Union, Optional
from datetime import datetime

import os, re, json, time
import typing as T
import requests
from collections import defaultdict
from pathlib import Path
from openpyxl import Workbook, load_workbook

from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from data_processing import load_and_process,build_jsonl_for_range, save_jsonl
from model_classifyV1_Copy1_Copy1 import add_index_to_jsonl_lines

In [4]:
## 研发字典
speaker_map = {
    "16186514":   "peter本尊",
    "1655611808": "运营绾绾",
    "2073820674": "沙利文老师",
    "2726067525": "milissa",
}
## 客服字典
MAPPING_FILE = "mapping地球1.xlsx"

##QQ的txt文件
pathtxt   = "0107《欢迎来到地球》测试2群.txt"

# 设定时间范围
start_time = "2025-12-08 00:00:00"
end_time   = "2025-12-09 00:00:00"


# 1) 拿到 JSONL（列表）
jsonl_lines01 = build_jsonl_for_range(
    pathtxt=pathtxt,
    mapping_file=MAPPING_FILE,
    speaker_map=speaker_map,
    start_time=start_time,
    end_time=end_time,
    return_str=False,   # 返回 list[str]
)
print(len(jsonl_lines01), "lines")
print(jsonl_lines01[0])



1270 lines
{"发言日期": "2025-12-08", "发言时间": "00:06:54", "玩家ID": "你吹你的牛 我抱我的熊(3100704086)", "玩家消息": "[图片]"}


## 大模型分类

## 定义

In [5]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：提取高讨论的发言并分析
- 结果按word格式文档输出
"""
from model_classifyV1_Copy1_Copy1 import (
    load_system_prompt,
    build_user_prompt_filter,build_user_prompt_clsuter,call_ark_chat_completions,
    extract_valid_json_lines,add_index_to_jsonl_lines,count_output_filter_stats,get_covered_indices_from_cluster_output,
    aggregate_cluster_outputs,build_user_prompt_cluster_agg,assign_global_cluster_ids,
    extract_top5_heat_clusters,attach_discussion_points_day,attach_discussion_points_all,extract_cluster_stats,append_daily_top5_to_version_jsonl,infer_date_for_batch,
    build_user_prompt_subcluster_opinion,
    print_mech_time_from_top5,get_dialogs_lines_by_fayan_time_debug,merge_top5_with_opinions_numbered,parse_opinion_output_to_list,
    ensure_time_axis_key,ensure_subcluster_list_key,match_dialogs_by_time, parse_jsonl_text,parse_model2_output_to_json_list,parse_jsonl_text_safe,
    normalize_model3_clusters,enrich_subclusters_with_datetime,refill_cluster_fayan_time
)

## 设置参数

In [6]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions" 
API_KEY = "de91deb0-aae6-46cb-bac0-17ac3b6107f5" #API
V3_MODEL_ID= "ep-20251020160142-5d7hp"#接入点
V3_1_MODEL_ID = "ep-20251020160025-9p5tj"#接入点
R1_MODEL_ID = "ep-20251020160103-5n6g2"#接入点

PROMPT_MD_PATH01 = Path("提示词1.md") # 模型#1 system 提示词（筛相关）
PROMPT_MD_PATH02 = Path("2话题分类.md") # 模型#2 system 提示词（分话题）
PROMPT_MD_PATH03 = Path("3日聚合.md") # 模型#3 system 提示词（日聚合）
PROMPT_MD_PATH04 = Path("2话题分类和总结.md") #模型#4 system 提示词（观点分析）
VERSION_TOP5_JSONL_day = "12311群.jsonl"
VERSION_TOP5_JSONL_all = "test_top5.jsonl"
BATCH_SIZE       = 300
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.20
MAX_TOKENS       = 16384
TIMEOUT_SEC      = 600
# =====================================================

## 1.检验每日话提簇分类输出

## 加讨论观点分析的版本测试

In [ ]:
from tqdm import tqdm
import time
import json
from pathlib import Path
from collections import defaultdict

# --- 前置：系统提示、初始化等 ---
system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话提簇
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 日话题簇聚合
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)  # 玩家观点分析（模型4）

batch_cluster_outputs = []   # 模型2所有批次簇，用于日聚合
all_opinions = []            # 模型4观点结果汇总

total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

# ====================================================
# 模型 #1 + 模型 #2：筛相关 + 话题簇划分
# ====================================================
for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue

        filter_count = sum(1 for line in output_filter.splitlines() if line.strip())
        tqdm.write(f"[批次 {b+1}] 模型#1 筛后保留 {filter_count} 条")
        written_total += filter_count

        # --- 模型 #2：话题簇划分 ---
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )

        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue
      
       # ---------- ✅ 新的 JSON 解析 + _cluster_id 部分 ----------
        try:
            # 1) 先用修复解析器，把模型#2输出变成 list[dict]
            cluster_json_list = parse_model2_output_to_json_list(
                output_cluster,
                batch_idx=b+1,
            )

            if not cluster_json_list:
                tqdm.write(f"[批次 {b+1}] ⚠ 本批次未解析出任何合法 JSON 对象，跳过。")
                continue
            
            
            # 2) 从该批次中推断日期（你自己的逻辑）
            date_str = infer_date_for_batch(cluster_json_list, batch_lines)
            batch_id = f"B{b+1}"

            # 3) 分配全局 _cluster_id：日期 + 批次号 + 序号 ...
            cluster_json_list = assign_global_cluster_ids(
                cluster_json_list,
                date_str,
                batch_id,
            )

            # 4) 再转成 jsonl 文本（供后续 aggregate_cluster_outputs 使用）
            output_cluster_with_ids = "\n".join(
                json.dumps(c, ensure_ascii=False) for c in cluster_json_list
            )
            batch_cluster_outputs.append(output_cluster_with_ids)
           
        except Exception as e:
            tqdm.write(f"[批次 {b+1}] ⚠ 添加 _cluster_id 失败：{e}")
            continue
        # ---------- JSON 解析部分结束 ----------

    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)
# ====================================================
# 模型 #3：日话题簇聚合 & Top5
# ====================================================

# --- 聚合当天所有批次的话题簇 jsonl ---
all_cluster = aggregate_cluster_outputs(batch_cluster_outputs)

# --- 模型 #3：日话题簇聚合 ---
user_prompt3 = build_user_prompt_cluster_agg(all_cluster)
output_cluster_agg = call_ark_chat_completions(
    api_url=API_URL,
    api_key=API_KEY,
    model=V3_1_MODEL_ID,
    system_prompt=system_prompt03,
    user_prompt=user_prompt3,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=TIMEOUT_SEC,
    retries=RETRIES,
)

# 1) 先解析子话题簇
parsed_subclusters = parse_jsonl_text(all_cluster)
parsed_clusters, parsed_subclusters_enriched = normalize_model3_clusters(output_cluster_agg, parsed_subclusters)



# 兜底修 key（时间轴 / 子话题簇列表）
fixed_cnt = 0
fixed_sub_cnt = 0
for c in parsed_clusters:
    if ensure_time_axis_key(c, verbose=False):
        fixed_cnt += 1
    if ensure_subcluster_list_key(c):
        fixed_sub_cnt += 1
print(f"✅ 已自动补齐/修复 时间轴 的聚合簇数量：{fixed_cnt}")
print(f"✅ 已自动补齐/修复 子话题簇列表 的聚合簇数量：{fixed_sub_cnt}")

# 计算每日讨论热度 Top5
top5_results = extract_top5_heat_clusters(parsed_clusters, jsonl_lines01, top_k=5)
# 4) attach 讨论点：日版建议用“已补齐”的 parsed_subclusters
final_result_day = attach_discussion_points_day(top5_results, parsed_subclusters)

# 5) 全版本版用你 normalize 返回的 enriched（也可以）
final_result_all = attach_discussion_points_all(top5_results, parsed_subclusters_enriched)

# ====================================================
# 模型 #4：根据 Top5 回溯原文，做观点共识分析
# ====================================================

rows = print_mech_time_from_top5(final_result_day, all_cluster)  # 输出含 核心对象/机制 + 发言时间

all_opinions = []

for idx, r in enumerate(rows, start=1):
    mech = r.get("核心对象/机制") or ""
    full_time = (r.get("发言时间") or "").strip()

    if not mech or not full_time:
        print(f"⚠ 第 {idx} 条缺少 核心对象/机制 或 发言时间，原始行：{r}")
        continue

    # full_time 形如："2025-12-02 14:01:57-14:42:21"
    if " " not in full_time:
        print(f"⚠ 第 {idx} 条发言时间格式异常，无法拆分日期和时间轴：{full_time}")
        continue

    fayan_date, fayan_time = full_time.split(" ", 1)

    dialogs_lines = get_dialogs_lines_by_fayan_time_debug(
        jsonl_lines01,
        fayan_date,
        fayan_time,
        debug=True,
    )

    if not dialogs_lines:
        print(f"⚠ 该时间段内没有原文，跳过：{mech} | {fayan_date} {fayan_time}")
        continue

    # --- 模型 #4：分析玩家共识 ---
    user_prompt4 = build_user_prompt_subcluster_opinion(
        discussion_point=mech,
        json_lines=dialogs_lines,
    )

    try:
        opinion_output = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt04,
            user_prompt=user_prompt4,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
    except Exception as e:
        print("❌ 调用模型4出错:", e)
        continue

    opinions_this_mech = parse_opinion_output_to_list(opinion_output)

    # ⭐ 强制把当前 discussion_point 写回每个 opinion 的「讨论点」字段
    for op in opinions_this_mech:
        op["讨论点"] = mech
    
    all_opinions.extend(opinions_this_mech)

# 合并模型4观点结果回 Top5
merged_top5 = merge_top5_with_opinions_numbered(final_result_day, all_opinions)

for row in merged_top5:
    print(json.dumps(row, ensure_ascii=False, indent=2))

#如要把带观点版写入版本 jsonl，可以用 merged_top5；如果只要热度+讨论点，就用 final_result_day
append_daily_top5_to_version_jsonl(
merged_top5,
version_jsonl_path=VERSION_TOP5_JSONL_day
)
#整个版本
append_daily_top5_to_version_jsonl(
final_result_all,
version_jsonl_path=VERSION_TOP5_JSONL_all
)
         




准备处理 1270 条，共 5 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                                               | 0/5 [00:00<?, ?批/s]

In [23]:
rows

[{'核心对象/机制': '背包重新排列后的殖装搭配策略优化', '发言时间': '2026-01-01 09:27:19-09:27:19'},
 {'核心对象/机制': '锤子殖装在四图的获取方式与地鼠殖装搭配问题', '发言时间': '2026-01-01 09:27:55-09:29:36'},
 {'核心对象/机制': '运动器材类殖装（哑铃等）的搭配策略与使用效果', '发言时间': '2026-01-01 09:35:35-09:42:53'},
 {'核心对象/机制': '精神病殖装的配置方案与等级要求（15级前后差异）',
  '发言时间': '2026-01-01 09:41:39-09:57:39'},
 {'核心对象/机制': '开局角色选择强度问题，涉及肿头龙、蝴蝶等具体殖装的强度对比',
  '发言时间': '2026-01-01 15:45:18-15:52:57'},
 {'核心对象/机制': '闪蝶殖装的实战性能评估与培养成本分析', '发言时间': '2026-01-01 16:52:40-16:56:33'},
 {'核心对象/机制': '秃鹫、土拨鼠等殖装的前期套装搭配效果与推荐', '发言时间': '2026-01-01 17:59:22-18:00:31'},
 {'核心对象/机制': '不同殖装装备的性能比较与选择建议', '发言时间': '2026-01-01 22:23:04-22:25:06'},
 {'核心对象/机制': '1-4号位殖装分工、投掷物使用与BOSS战阵容搭配',
  '发言时间': '2026-01-01 22:32:01-22:51:43'},
 {'核心对象/机制': '游戏重启机制对见闻、殖装等级、积分点数的保留规则',
  '发言时间': '2026-01-01 08:13:25-08:26:12'},
 {'核心对象/机制': '999魔方获取条件、重启后魔方S等级清空规则、高要求殖装开局限制',
  '发言时间': '2026-01-01 08:51:07-09:04:41'},
 {'核心对象/机制': '殖装下阵限制与重启周目机制说明', '发言时间': '2026-01-01 10:29:09-10:34:15'},
 {'核心对象/机制': '重启周目时殖装装备的保留规则与异常